In [7]:
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
from enum import Enum
import tensorflow as tf

np.random.seed(1)

In [8]:
class ActivationType(Enum):
  Relu = lambda x: np.max(X, 0)
  Softmax = lambda x: np.exp(x)/sum(np.exp(x))
  Sigmoid = lambda x: 1 / (1 + np.exp(-x))

In [9]:
def forward_propagation(X, W, b, g):
  # print("X",X.shape,"W", W.shape)
  z = np.dot(X, W) + b
  a = g(z)
  return z,a


In [10]:
def sigmoid_gradient(self, z):
  a = self._sigmoid(z)
  return a * (1 - a)


In [11]:
def backward_propagation(params, curr_layer, Y, m, last_layer):
  A_prev = params["A"+str(curr_layer - 1)]
  
  dZ = []
  if curr_layer == last_layer:
    # print("****** A{} = {}".format(curr_layer, params["A"+str(curr_layer)].shape) )
    dZ = params["A"+str(curr_layer)] - Y
  else:
    # print("W{} = {} \t dZ{} = {}".format(curr_layer, A_prev.shape, curr_layer, dZ.shape))
    dZ = np.dot(params['dZ' + str(curr_layer + 1)], params['W' + str(curr_layer + 1)].T) 
  # print("A{} = {} \t Z{} = {}".format(curr_layer, A_prev.shape, curr_layer, dZ.shape))
  dW = 1/m * np.dot( A_prev.T, dZ )
  db = 1/m * np.sum(dZ, axis=0, keepdims=True)
  return dZ, dW, db


In [21]:
def calculate_cost(params, Y, m):
  Y = np.array(Y)
  cost = -(1/m) * np.sum(Y * np.log(params["A4"]) + (1-Y) * np.log(1-params["A4"]))
  return cost


In [19]:
def fc(input, layer_dim: list, action: ActivationType, Y, m):
  parameters = {}
  X = input.reshape(input.shape[0],-1)
  dim = layer_dim
  dim.insert(0, X.shape[1])
  parameters["A0"] = X
  for l in range(1, len(dim)):
    parameters["W"+str(l)] = np.random.randn(dim[l-1], dim[l]) * 0.01
    parameters["b"+str(l)] = np.zeros((1, dim[l]))

    Z, A = forward_propagation(parameters["A"+str(l-1)], parameters["W"+str(l)], parameters["b"+str(l)], ActivationType.Sigmoid)
    parameters["Z"+str(l)] = Z
    parameters["A"+str(l)] = A
    print("{} = W -> {}\t b -> {}\t z -> {}\t A -> {}".format(l, parameters["W"+str(l)].shape, parameters["b"+str(l)].shape, parameters["Z"+str(l)].shape, parameters["A"+str(l)].shape))
    
  for l in range(len(dim) - 1, 0, -1):
    # print("reversed L ", l)
    dZ, dW, db = backward_propagation(parameters, l, Y, m, len(dim)-1)
    parameters["dZ"+str(l)] = dZ
    parameters["dW"+str(l)] = dW
    parameters["db"+str(l)] = db
    # print("dZ{} = {}\t dW{} = {}\t ".format(l, parameters["dZ"+str(l)].shape, l, parameters["dW"+str(l)].shape))
  
  for i in range(1, len(dim)):
    print("{} = dW -> {}\t db -> {}\t dz -> {}\t A -> {}".format(i, parameters["dW"+str(i)].shape, parameters["db"+str(i)].shape, parameters["dZ"+str(i)].shape, parameters["A"+str(i)].shape))
    parameters["W"+str(i)] -= 0.01 * parameters["dW"+str(i)]
    parameters["b"+str(i)] -= 0.01 * parameters["db"+str(i)]

  J = calculate_cost(parameters, Y, m)
  print("cost >>> ", J)
  
  
  # print("Keys ",parameters.keys())

In [22]:
image = np.random.randn(10,64,64,3)

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train / 255
x_test = x_test / 255

Y = np.zeros((y_train.shape[0], 10))
for idnx in range(y_train.shape[0]):
  active_indx = y_train[idnx]
  Y[idnx][active_indx] = 1
print( y_train.shape, np.unique(y_train), y_train[1])
for i in range(100):
  fc(x_train, [40,30,20,10], ActivationType.Softmax, Y, x_train.shape[0])

(60000,) [0 1 2 3 4 5 6 7 8 9] 0
1 = W -> (784, 40)	 b -> (1, 40)	 z -> (60000, 40)	 A -> (60000, 40)
2 = W -> (40, 30)	 b -> (1, 30)	 z -> (60000, 30)	 A -> (60000, 30)
3 = W -> (30, 20)	 b -> (1, 20)	 z -> (60000, 20)	 A -> (60000, 20)
4 = W -> (20, 10)	 b -> (1, 10)	 z -> (60000, 10)	 A -> (60000, 10)
1 = dW -> (784, 40)	 db -> (1, 40)	 dz -> (60000, 40)	 A -> (60000, 40)
2 = dW -> (40, 30)	 db -> (1, 30)	 dz -> (60000, 30)	 A -> (60000, 30)
3 = dW -> (30, 20)	 db -> (1, 20)	 dz -> (60000, 20)	 A -> (60000, 20)
4 = dW -> (20, 10)	 db -> (1, 10)	 dz -> (60000, 10)	 A -> (60000, 10)
cost >>>  6.9199779716291125
1 = W -> (784, 40)	 b -> (1, 40)	 z -> (60000, 40)	 A -> (60000, 40)
2 = W -> (40, 30)	 b -> (1, 30)	 z -> (60000, 30)	 A -> (60000, 30)
3 = W -> (30, 20)	 b -> (1, 20)	 z -> (60000, 20)	 A -> (60000, 20)
4 = W -> (20, 10)	 b -> (1, 10)	 z -> (60000, 10)	 A -> (60000, 10)
1 = dW -> (784, 40)	 db -> (1, 40)	 dz -> (60000, 40)	 A -> (60000, 40)
2 = dW -> (40, 30)	 db -> (1, 30)	 